In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler, random_split
import torchvision
from sklearn.model_selection import train_test_split, KFold
from matplotlib import pyplot as plt
from torchvision import transforms, datasets
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
import os
from pathlib import Path
import numpy as np
import torch.backends.cudnn as cudnn

cudnn.benchmark = True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
data_dir = 'data/train'

In [4]:
#weights = torchvision.models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
weights = torchvision.models.EfficientNet_V2_L_Weights.IMAGENET1K_V1
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.5, 0.5, 0.5]
    std=[0.5, 0.5, 0.5]
    interpolation=InterpolationMode.BICUBIC
)

In [5]:
adjusted_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(20),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.Lambda(lambda img: auto_transforms(img))
])
adjusted_transforms

Compose(
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    Lambda()
)

In [ ]:
from tools.data_prep import create_dataloaders

train_loader, test_loader, class_names = create_dataloaders(path=data_dir, train_transform=adjusted_transforms, test_transform=auto_transforms, batch_size=16, train_split=0.8, seed=4422)
model = torchvision.models.efficientnet_v2_l(weights=weights)

num_classes = len(class_names)

#summary(model, input_size=(16, 3, 480, 480), col_names=["input_size", "output_size", "num_params", "trainable"])

In [ ]:
from tools.engine import train
from tools.engine import sam_train
from tools.test_tracking import create_writer
from tools.sam import SAM

torch.manual_seed(42)
torch.cuda.manual_seed(42)

for param in model.parameters():
    param.requires_grad = False

if hasattr(model, 'features') and isinstance(model.features, torch.nn.Sequential):
    for param in model.features[-2:].parameters():
        param.requires_grad = True

model.classifier = nn.Sequential(
    nn.Dropout(p=0.25, inplace=False),
    nn.Linear(1280, num_classes)
).to(device)

# Enable gradients for the new classifier layer
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
summary(model, input_size=(16, 3, 480, 480), col_names=["trainable"])

In [ ]:

loss_fn = nn.CrossEntropyLoss()
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, lr=0.004, rho=1.75, momentum=0.9)

writer = create_writer('FINAL-ATTEMPT')
save_path = 'models/FINAL-ATTEMPT'

model.to(device)

sam_train(model=model, train_loader=train_loader, test_loader=test_loader, loss_fn=loss_fn, optimizer=optimizer, num_epochs=1000, writer=writer, save_path=save_path)
